In [6]:
import pandas as pd
from create_db import engine
from sqlalchemy.exc import ProgrammingError
import warnings
import numpy as np
import requests
import datetime
import blackcellmagic

In [7]:

DATABASE = "team_big_bend"
engine.execute(f"USE {DATABASE}")

In [8]:
TABLENAME = "bluetooth"
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME}")

In [9]:
#Set up the api url and parameters
bluetooth_raw = []

limit=50000
offset=0

while True:
    r = requests.get("https://data.austintexas.gov/resource/x44q-icha.json?$where=origin_reader_identifier like '%amar_%'"
    , params = {
    "$limit": limit,
    "$offset": offset
            })
    if r.status_code == 200:
        bluetooth_raw = [{
        "ID": rec["record_id"],
        "Start": rec["origin_reader_identifier"].upper(),
        "Ending": rec["destination_reader_identifier"].upper(),
        "TravelTime": int(rec["travel_time_seconds"]),
        "MPH": int(rec["speed_miles_per_hour"]),
        "StartTime": rec["start_time"],
        "EndTime": rec["end_time"]
        } for rec in r.json()]
        
        #bluetooth_raw = r.json()
    else:
        print("***** IT IS BROKEN *****")
        print(r.status_code)
        break
    bluetooth_raw = pd.DataFrame(bluetooth_raw)
    if bluetooth_raw.empty:
        break
    pd.DataFrame(bluetooth_raw).to_sql(name=TABLENAME, con=engine, if_exists='append')
    offset=offset+limit